# 🚀 SIMPLE ULTIMATE BULLETPROOF EMOTION DETECTION TRAINING
## Avoiding Datasets Library Issues

**FEATURES INCLUDED:**
✅ Configuration preservation (prevents 8.3% vs 75% discrepancy)
✅ Focal loss (handles class imbalance)
✅ Class weighting (WeightedLossTrainer)
✅ Data augmentation (sophisticated techniques)
✅ Advanced validation (proper testing)
✅ Simple, direct approach (no datasets library issues)

**Target**: Reliable 75-85% F1 score with consistent performance

In [ ]:
# Install required packages
!pip install transformers torch scikit-learn numpy pandas huggingface_hub

In [ ]:
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score, precision_score, recall_score
from sklearn.utils.class_weight import compute_class_weight
import json
import warnings
warnings.filterwarnings('ignore')

print('✅ All packages imported successfully')
print(f'PyTorch version: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')

## 🔍 VERIFYING SPECIALIZED MODEL ACCESS

In [ ]:
print('🔍 VERIFYING SPECIALIZED MODEL ACCESS')
print('=' * 50)

specialized_model_name = 'j-hartmann/emotion-english-distilroberta-base'

try:
    print(f'Testing access to: {specialized_model_name}')
    test_tokenizer = AutoTokenizer.from_pretrained(specialized_model_name)
    test_model = AutoModelForSequenceClassification.from_pretrained(specialized_model_name)
    
    print('✅ SUCCESS: Specialized model loaded!')
    print(f'Model type: {test_model.config.model_type}')
    print(f'Architecture: {test_model.config.architectures[0]}')
    print(f'Hidden layers: {test_model.config.num_hidden_layers}')
    print(f'Hidden size: {test_model.config.hidden_size}')
    print(f'Number of labels: {test_model.config.num_labels}')
    print(f'Original labels: {test_model.config.id2label}')
    
    # Verify it's actually DistilRoBERTa
    if test_model.config.num_hidden_layers == 6:
        print('✅ CONFIRMED: This is DistilRoBERTa architecture')
    else:
        print('⚠️  WARNING: This may not be the expected DistilRoBERTa model')
    
except Exception as e:
    print(f'❌ ERROR: Cannot access specialized model: {str(e)}')
    print('\n🔧 FALLBACK: Using roberta-base instead')
    specialized_model_name = 'roberta-base'
    test_tokenizer = AutoTokenizer.from_pretrained(specialized_model_name)
    test_model = AutoModelForSequenceClassification.from_pretrained(specialized_model_name, num_labels=12)
    print(f'✅ Fallback model loaded: {specialized_model_name}')

## 🎯 DEFINING EMOTION CLASSES

In [ ]:
# Define our emotion classes
emotions = ['anxious', 'calm', 'content', 'excited', 'frustrated', 'grateful', 'happy', 'hopeful', 'overwhelmed', 'proud', 'sad', 'tired']
print(f'🎯 Our emotion classes: {emotions}')
print(f'📊 Number of emotions: {len(emotions)}')

## 📊 CREATING ENHANCED DATASET WITH AUGMENTATION

In [ ]:
print('📊 CREATING ENHANCED DATASET WITH AUGMENTATION')
print('=' * 50)

# Base balanced dataset
base_data = [
    # anxious (12 samples)
    {'text': 'I feel anxious about the presentation.', 'label': 0},
    {'text': 'I am anxious about the future.', 'label': 0},
    {'text': 'This makes me feel anxious.', 'label': 0},
    {'text': 'I am feeling anxious today.', 'label': 0},
    {'text': 'The uncertainty makes me anxious.', 'label': 0},
    {'text': 'I feel anxious about the results.', 'label': 0},
    {'text': 'This situation is making me anxious.', 'label': 0},
    {'text': 'I am anxious about the meeting.', 'label': 0},
    {'text': 'The pressure is making me anxious.', 'label': 0},
    {'text': 'I feel anxious about the decision.', 'label': 0},
    {'text': 'This is causing me anxiety.', 'label': 0},
    {'text': 'I am anxious about the changes.', 'label': 0},
    
    # calm (12 samples)
    {'text': 'I feel calm and peaceful.', 'label': 1},
    {'text': 'I am feeling calm today.', 'label': 1},
    {'text': 'This makes me feel calm.', 'label': 1},
    {'text': 'I am calm about the situation.', 'label': 1},
    {'text': 'I feel calm and relaxed.', 'label': 1},
    {'text': 'This gives me a sense of calm.', 'label': 1},
    {'text': 'I am feeling calm and centered.', 'label': 1},
    {'text': 'This brings me calm.', 'label': 1},
    {'text': 'I feel calm and at peace.', 'label': 1},
    {'text': 'I am calm about the outcome.', 'label': 1},
    {'text': 'This creates a feeling of calm.', 'label': 1},
    {'text': 'I feel calm and collected.', 'label': 1},
    
    # content (12 samples)
    {'text': 'I feel content with my life.', 'label': 2},
    {'text': 'I am content with the results.', 'label': 2},
    {'text': 'This makes me feel content.', 'label': 2},
    {'text': 'I am feeling content today.', 'label': 2},
    {'text': 'I feel content and satisfied.', 'label': 2},
    {'text': 'This gives me contentment.', 'label': 2},
    {'text': 'I am content with my choices.', 'label': 2},
    {'text': 'I feel content and fulfilled.', 'label': 2},
    {'text': 'This brings me contentment.', 'label': 2},
    {'text': 'I am content with the situation.', 'label': 2},
    {'text': 'I feel content and at ease.', 'label': 2},
    {'text': 'This creates contentment in me.', 'label': 2},
    
    # excited (12 samples)
    {'text': 'I am excited about the new opportunity.', 'label': 3},
    {'text': 'I feel excited about the future.', 'label': 3},
    {'text': 'This makes me feel excited.', 'label': 3},
    {'text': 'I am feeling excited today.', 'label': 3},
    {'text': 'I feel excited and enthusiastic.', 'label': 3},
    {'text': 'This gives me excitement.', 'label': 3},
    {'text': 'I am excited about the project.', 'label': 3},
    {'text': 'I feel excited and motivated.', 'label': 3},
    {'text': 'This brings me excitement.', 'label': 3},
    {'text': 'I am excited about the possibilities.', 'label': 3},
    {'text': 'I feel excited and energized.', 'label': 3},
    {'text': 'This creates excitement in me.', 'label': 3},
    
    # frustrated (12 samples)
    {'text': 'I am so frustrated with this project.', 'label': 4},
    {'text': 'I feel frustrated about the situation.', 'label': 4},
    {'text': 'This makes me feel frustrated.', 'label': 4},
    {'text': 'I am feeling frustrated today.', 'label': 4},
    {'text': 'I feel frustrated and annoyed.', 'label': 4},
    {'text': 'This gives me frustration.', 'label': 4},
    {'text': 'I am frustrated with the results.', 'label': 4},
    {'text': 'I feel frustrated and irritated.', 'label': 4},
    {'text': 'This brings me frustration.', 'label': 4},
    {'text': 'I am frustrated with the process.', 'label': 4},
    {'text': 'I feel frustrated and upset.', 'label': 4},
    {'text': 'This creates frustration in me.', 'label': 4},
    
    # grateful (12 samples)
    {'text': 'I am grateful for all the support.', 'label': 5},
    {'text': 'I feel grateful for the opportunity.', 'label': 5},
    {'text': 'This makes me feel grateful.', 'label': 5},
    {'text': 'I am feeling grateful today.', 'label': 5},
    {'text': 'I feel grateful and thankful.', 'label': 5},
    {'text': 'This gives me gratitude.', 'label': 5},
    {'text': 'I am grateful for the help.', 'label': 5},
    {'text': 'I feel grateful and appreciative.', 'label': 5},
    {'text': 'This brings me gratitude.', 'label': 5},
    {'text': 'I am grateful for the kindness.', 'label': 5},
    {'text': 'I feel grateful and blessed.', 'label': 5},
    {'text': 'This creates gratitude in me.', 'label': 5},
    
    # happy (12 samples)
    {'text': 'I am feeling really happy today!', 'label': 6},
    {'text': 'I feel happy about the news.', 'label': 6},
    {'text': 'This makes me feel happy.', 'label': 6},
    {'text': 'I am feeling happy today.', 'label': 6},
    {'text': 'I feel happy and joyful.', 'label': 6},
    {'text': 'This gives me happiness.', 'label': 6},
    {'text': 'I am happy with the results.', 'label': 6},
    {'text': 'I feel happy and delighted.', 'label': 6},
    {'text': 'This brings me happiness.', 'label': 6},
    {'text': 'I am happy about the success.', 'label': 6},
    {'text': 'I feel happy and cheerful.', 'label': 6},
    {'text': 'This creates happiness in me.', 'label': 6},
    
    # hopeful (12 samples)
    {'text': 'I am hopeful for the future.', 'label': 7},
    {'text': 'I feel hopeful about the outcome.', 'label': 7},
    {'text': 'This makes me feel hopeful.', 'label': 7},
    {'text': 'I am feeling hopeful today.', 'label': 7},
    {'text': 'I feel hopeful and optimistic.', 'label': 7},
    {'text': 'This gives me hope.', 'label': 7},
    {'text': 'I am hopeful about the changes.', 'label': 7},
    {'text': 'I feel hopeful and positive.', 'label': 7},
    {'text': 'This brings me hope.', 'label': 7},
    {'text': 'I am hopeful about the possibilities.', 'label': 7},
    {'text': 'I feel hopeful and confident.', 'label': 7},
    {'text': 'This creates hope in me.', 'label': 7},
    
    # overwhelmed (12 samples)
    {'text': 'I am feeling overwhelmed with tasks.', 'label': 8},
    {'text': 'I feel overwhelmed by the workload.', 'label': 8},
    {'text': 'This makes me feel overwhelmed.', 'label': 8},
    {'text': 'I am feeling overwhelmed today.', 'label': 8},
    {'text': 'I feel overwhelmed and stressed.', 'label': 8},
    {'text': 'This gives me overwhelm.', 'label': 8},
    {'text': 'I am overwhelmed with responsibilities.', 'label': 8},
    {'text': 'I feel overwhelmed and exhausted.', 'label': 8},
    {'text': 'This brings me overwhelm.', 'label': 8},
    {'text': 'I am overwhelmed with the pressure.', 'label': 8},
    {'text': 'I feel overwhelmed and drained.', 'label': 8},
    {'text': 'This creates overwhelm in me.', 'label': 8},
    
    # proud (12 samples)
    {'text': 'I am proud of my accomplishments.', 'label': 9},
    {'text': 'I feel proud of the results.', 'label': 9},
    {'text': 'This makes me feel proud.', 'label': 9},
    {'text': 'I am feeling proud today.', 'label': 9},
    {'text': 'I feel proud and accomplished.', 'label': 9},
    {'text': 'This gives me pride.', 'label': 9},
    {'text': 'I am proud of my achievements.', 'label': 9},
    {'text': 'I feel proud and satisfied.', 'label': 9},
    {'text': 'This brings me pride.', 'label': 9},
    {'text': 'I am proud of my progress.', 'label': 9},
    {'text': 'I feel proud and confident.', 'label': 9},
    {'text': 'This creates pride in me.', 'label': 9},
    
    # sad (12 samples)
    {'text': 'I feel sad about the loss.', 'label': 10},
    {'text': 'I am sad about the situation.', 'label': 10},
    {'text': 'This makes me feel sad.', 'label': 10},
    {'text': 'I am feeling sad today.', 'label': 10},
    {'text': 'I feel sad and down.', 'label': 10},
    {'text': 'This gives me sadness.', 'label': 10},
    {'text': 'I am sad about the outcome.', 'label': 10},
    {'text': 'I feel sad and depressed.', 'label': 10},
    {'text': 'This brings me sadness.', 'label': 10},
    {'text': 'I am sad about the news.', 'label': 10},
    {'text': 'I feel sad and heartbroken.', 'label': 10},
    {'text': 'This creates sadness in me.', 'label': 10},
    
    # tired (12 samples)
    {'text': 'I am tired from working all day.', 'label': 11},
    {'text': 'I feel tired of the routine.', 'label': 11},
    {'text': 'This makes me feel tired.', 'label': 11},
    {'text': 'I am feeling tired today.', 'label': 11},
    {'text': 'I feel tired and exhausted.', 'label': 11},
    {'text': 'This gives me fatigue.', 'label': 11},
    {'text': 'I am tired of the stress.', 'label': 11},
    {'text': 'I feel tired and worn out.', 'label': 11},
    {'text': 'This brings me fatigue.', 'label': 11},
    {'text': 'I am tired of the pressure.', 'label': 11},
    {'text': 'I feel tired and drained.', 'label': 11},
    {'text': 'This creates fatigue in me.', 'label': 11}
]

print(f'📊 Base dataset size: {len(base_data)} samples')

# Data augmentation function
def augment_text(text, emotion):
    """Create augmented versions of the text."""
    augmented = []
    
    # Synonym replacement
    synonyms = {
        'anxious': ['worried', 'nervous', 'concerned', 'uneasy'],
        'calm': ['peaceful', 'serene', 'tranquil', 'relaxed'],
        'content': ['satisfied', 'fulfilled', 'pleased', 'happy'],
        'excited': ['thrilled', 'enthusiastic', 'eager', 'pumped'],
        'frustrated': ['annoyed', 'irritated', 'aggravated', 'bothered'],
        'grateful': ['thankful', 'appreciative', 'blessed', 'indebted'],
        'happy': ['joyful', 'cheerful', 'delighted', 'pleased'],
        'hopeful': ['optimistic', 'positive', 'confident', 'assured'],
        'overwhelmed': ['stressed', 'burdened', 'swamped', 'flooded'],
        'proud': ['accomplished', 'satisfied', 'confident', 'pleased'],
        'sad': ['down', 'depressed', 'melancholy', 'blue'],
        'tired': ['exhausted', 'fatigued', 'weary', 'drained']
    }
    
    # Create variations with synonyms
    for synonym in synonyms.get(emotion, [emotion])[:2]:  # Use first 2 synonyms
        new_text = text.replace(emotion, synonym)
        if new_text != text:
            augmented.append({'text': new_text, 'label': emotions.index(emotion)})
    
    # Add intensity variations
    intensity_words = ['really', 'very', 'extremely', 'quite', 'somewhat']
    for intensity in intensity_words[:2]:
        if intensity not in text.lower():
            new_text = f'I am {intensity} {emotion}.'
            augmented.append({'text': new_text, 'label': emotions.index(emotion)})
    
    return augmented

# Apply augmentation
augmented_data = []
for item in base_data:
    emotion = emotions[item['label']]
    augmented = augment_text(item['text'], emotion)
    augmented_data.extend(augmented)

# Combine base and augmented data
enhanced_data = base_data + augmented_data
print(f'📊 Enhanced dataset size: {len(enhanced_data)} samples')
print(f'📊 Augmentation added: {len(augmented_data)} samples')

# Convert to lists for simple processing
texts = [item['text'] for item in enhanced_data]
labels = [item['label'] for item in enhanced_data]

print(f'✅ Dataset prepared with {len(texts)} samples')

## 🎯 FOCAL LOSS IMPLEMENTATION

In [ ]:
# Focal Loss Implementation
class FocalLoss(torch.nn.Module):
    """Focal Loss for handling class imbalance."""
    
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
    
    def forward(self, inputs, targets):
        ce_loss = torch.nn.functional.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

print('✅ Focal Loss implementation ready')

## ⚖️ CLASS WEIGHTING & WEIGHTED LOSS TRAINER

In [ ]:
# Calculate class weights
print('⚖️ CALCULATING CLASS WEIGHTS')
print('=' * 40)

class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(labels),
    y=labels
)

class_weights_tensor = torch.FloatTensor(class_weights)
if torch.cuda.is_available():
    class_weights_tensor = class_weights_tensor.cuda()

print(f'Class weights: {class_weights}')
print(f'Class weights tensor shape: {class_weights_tensor.shape}')
print('✅ Class weights calculated')

# Weighted Loss Trainer
class WeightedLossTrainer(Trainer):
    """Custom trainer with focal loss and class weighting."""
    
    def __init__(self, focal_alpha=1, focal_gamma=2, class_weights=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.focal_loss = FocalLoss(alpha=focal_alpha, gamma=focal_gamma)
        self.class_weights = class_weights
    
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        
        # Apply focal loss with class weighting
        if self.class_weights is not None:
            # Apply class weights to focal loss
            ce_loss = torch.nn.functional.cross_entropy(logits, labels, reduction='none')
            pt = torch.exp(-ce_loss)
            focal_loss = (1 - pt) ** self.focal_loss.gamma * ce_loss
            
            # Apply class weights
            for i, weight in enumerate(self.class_weights):
                mask = (labels == i)
                focal_loss[mask] *= weight
            
            loss = focal_loss.mean()
        else:
            loss = self.focal_loss(logits, labels)
        
        return (loss, outputs) if return_outputs else loss

print('✅ WeightedLossTrainer ready')

## 🔧 LOADING & CONFIGURING MODEL

In [ ]:
# Load tokenizer and model
print('🔧 LOADING & CONFIGURING MODEL')
print('=' * 40)

tokenizer = AutoTokenizer.from_pretrained(specialized_model_name)
model = AutoModelForSequenceClassification.from_pretrained(specialized_model_name)

# Configure model for our emotion classes
model.config.num_labels = len(emotions)
model.config.id2label = {i: emotion for i, emotion in enumerate(emotions)}
model.config.label2id = {emotion: i for i, emotion in enumerate(emotions)}

# Verify configuration
print(f'✅ Model configured for {len(emotions)} emotions')
print(f'✅ id2label: {model.config.id2label}')
print(f'✅ label2id: {model.config.label2id}')

# Move to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
print(f'✅ Model moved to: {device}')

## 📝 DATA PREPROCESSING

In [ ]:
# Simple preprocessing without datasets library
print('📝 PREPROCESSING DATA')
print('=' * 40)

# Split data
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

print(f'📊 Training samples: {len(train_texts)}')
print(f'📊 Validation samples: {len(val_texts)}')

# Tokenize training data
train_encodings = tokenizer(
    train_texts,
    truncation=True,
    padding=True,
    max_length=128,
    return_tensors='pt'
)

# Tokenize validation data
val_encodings = tokenizer(
    val_texts,
    truncation=True,
    padding=True,
    max_length=128,
    return_tensors='pt'
)

# Create simple dataset class
class SimpleDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

# Create datasets
train_dataset = SimpleDataset(train_encodings, train_labels)
val_dataset = SimpleDataset(val_encodings, val_labels)

print('✅ Data preprocessing completed')

## ⚙️ TRAINING ARGUMENTS

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./ultimate_emotion_model',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_steps=50,
    save_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    greater_is_better=True,
    report_to='wandb',
    run_name='ultimate_emotion_model'
)

print('✅ Training arguments configured')

## 📊 COMPUTE METRICS

In [ ]:
# Compute metrics function
def compute_metrics(eval_pred):
    """Compute evaluation metrics."""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    return {
        'f1': f1_score(labels, predictions, average='weighted'),
        'accuracy': accuracy_score(labels, predictions),
        'precision': precision_score(labels, predictions, average='weighted'),
        'recall': recall_score(labels, predictions, average='weighted')
    }

print('✅ Compute metrics function ready')

## 🚀 TRAINING

In [ ]:
# Initialize trainer
trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    focal_alpha=1,
    focal_gamma=2,
    class_weights=class_weights_tensor
)

print('✅ Trainer initialized with focal loss and class weighting')

# Start training
print('🚀 STARTING ULTIMATE TRAINING')
print('=' * 50)
print(f'🎯 Target: 75-85% F1 score')
print(f'📊 Training samples: {len(train_dataset)}')
print(f'🧪 Validation samples: {len(val_dataset)}')
print(f'⚖️ Using focal loss + class weighting')
print(f'🔧 Model: {specialized_model_name}')

# Train the model
trainer.train()

print('✅ Training completed successfully!')

## 📈 EVALUATION

In [ ]:
# Evaluate the model
print('📈 EVALUATING MODEL')
print('=' * 40)

results = trainer.evaluate()
print('\n📊 FINAL RESULTS:')
print(f'F1 Score: {results["eval_f1"]:.4f}')
print(f'Accuracy: {results["eval_accuracy"]:.4f}')
print(f'Precision: {results["eval_precision"]:.4f}')
print(f'Recall: {results["eval_recall"]:.4f}')

print('✅ Evaluation completed!')

## 🧪 ADVANCED VALIDATION

In [ ]:
# Advanced validation on diverse examples
print('🧪 ADVANCED VALIDATION')
print('=' * 40)

# Test examples
test_examples = [
    'I am feeling anxious about the presentation tomorrow.',
    'I feel calm and peaceful after meditation.',
    'I am excited about the new job opportunity!',
    'I feel frustrated with the technical issues.',
    'I am grateful for all the support I received.',
    'I feel happy about the successful completion.',
    'I am hopeful for a better future.',
    'I feel overwhelmed with all the responsibilities.',
    'I am proud of my achievements.',
    'I feel sad about the recent loss.',
    'I am tired from working long hours.',
    'I feel content with my current situation.'
]

print('🔍 Testing on diverse examples:')
for i, example in enumerate(test_examples):
    inputs = tokenizer(example, return_tensors='pt', truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_class = torch.argmax(predictions, dim=-1).item()
        confidence = predictions[0][predicted_class].item()
    
    print(f'{i+1:2d}. "{example}" → {emotions[predicted_class]} ({confidence:.3f})')

print('✅ Advanced validation completed!')

## 💾 MODEL SAVING WITH VERIFICATION

In [ ]:
# Save model with verification
print('💾 SAVING MODEL WITH VERIFICATION')
print('=' * 50)

# Save the model
model_path = './ultimate_emotion_model_final'
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

print(f'✅ Model saved to: {model_path}')

# Verify the saved configuration
print('\n🔍 VERIFYING SAVED CONFIGURATION:')
config_path = f'{model_path}/config.json'
with open(config_path, 'r') as f:
    config = json.load(f)

print(f'Model type: {config.get("model_type", "NOT SET")}')
print(f'Number of labels: {config.get("num_labels", "NOT SET")}')
print(f'id2label: {config.get("id2label", "NOT SET")}')
print(f'label2id: {config.get("label2id", "NOT SET")}')

# Test loading the saved model
print('\n🧪 TESTING SAVED MODEL:')
test_tokenizer = AutoTokenizer.from_pretrained(model_path)
test_model = AutoModelForSequenceClassification.from_pretrained(model_path)

test_input = 'I feel happy about the results!'
test_encoding = test_tokenizer(test_input, return_tensors='pt', truncation=True, padding=True)
test_encoding = {k: v.to(device) for k, v in test_encoding.items()}

with torch.no_grad():
    test_outputs = test_model(**test_encoding)
    test_predictions = torch.nn.functional.softmax(test_outputs.logits, dim=-1)
    test_predicted_class = torch.argmax(test_predictions, dim=-1).item()
    test_confidence = test_predictions[0][test_predicted_class].item()

print(f'Test input: "{test_input}"')
print(f'Predicted emotion: {test_model.config.id2label[test_predicted_class]}')
print(f'Confidence: {test_confidence:.3f}')

print('\n✅ Model saving and verification completed!')